# RNN 기반 텍스트 분류기 (Text Classification)
### 과목: NLP Text Classification
---

## 1. 딥러닝으로 텍스트를 분류해보자
이전에는 나이브 베이즈 같은 통계적 방법을 썼다면, 이제는 **딥러닝(RNN/LSTM)**을 사용하여 분류해봅시다.
RNN은 문장의 **"순서 정보(Context)"**를 읽을 수 있어서, 문맥을 더 잘 파악할 수 있습니다.

**목표**:
1. 뉴스 기사 데이터를 읽어온다.
2. 컴퓨터가 이해하도록 **숫자(정수 인덱스)**로 바꾼다. (Tokenization)
3. **LSTM** 모델을 만들어 학습시킨다.
4. (심화) 이미 공부를 많이 한 **사전 학습 임베딩(Pre-trained Embedding)**을 가져와서 성능을 높여본다.

In [ ]:
# 1. 데이터 로드
# 20 Newsgroups 데이터셋 중 3가지 주제만 골라서 분류해봅시다.
from sklearn.datasets import fetch_20newsgroups

categories = ['comp.graphics', 'sci.space', 'rec.sport.baseball']
newsgroups = fetch_20newsgroups(subset='all', categories=categories)

X = newsgroups.data # 뉴스 본문 (리스트)
y = newsgroups.target # 정답 라벨 (0, 1, 2)

print(f"찾아야 할 주제들: {newsgroups.target_names}")
print(f"첫 번째 뉴스 본문 (앞 200자):\n{X[0][:200]}...")
print(f"첫 번째 정답 라벨: {y[0]} ({newsgroups.target_names[y[0]]})")

## 2. 텍스트 전처리 (Preprocessing)
딥러닝 모델은 글자를 못 읽습니다. 그래서 **숫자**로 바꿔줘야 합니다.
1. **Tokenizer**: 빈도수가 높은 단어 순서대로 번호(Index)를 매깁니다. (예: the -> 1, apple -> 2)
2. **Padding**: 모든 문장의 길이를 똑같이 맞춰줍니다. (짧으면 0을 채우고, 길면 자름)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 설정값들
vocab_size = 10000    # 사용할 단어 사전 크기 (가장 많이 쓰이는 10,000개만 사용)
max_len = 200         # 문장의 최대 길이 (200 단어)

# 1. 토크나이저 학습 (단어장 만들기)
# oov_token='<OOV>': 모르는 단어(사전에 없는 단어)는 '<OOV>'라고 표시함
tokenizer = Tokenizer(num_words=vocab_size, oov_token='<OOV>')
tokenizer.fit_on_texts(X)

# 2. 텍스트 -> 숫자 시퀀스 변환
X_encoded = tokenizer.texts_to_sequences(X)

# 3. 패딩 (길이 맞추기)
X_padded = pad_sequences(X_encoded, maxlen=max_len)

print(f"전체 데이터 크기 (문서 개수, 문장 길이): {X_padded.shape}")
# 결과: (문서개수, 200) -> 모든 문서가 200개 숫자로 된 리스트로 변함

In [ ]:
# 3. PyTorch용 데이터셋 만들기
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader

# Train / Validation / Test 분리 (6:2:2 비율)
X_train, X_test, y_train, y_test = train_test_split(torch.tensor(X_padded), torch.tensor(y), test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# DataLoader 생성 (배치 단위로 데이터를 묶어줌)
# TensorDataset: 입력(X)과 정답(y)을 묶어주는 보따리
# DataLoader: 보따리에서 데이터를 배치 사이즈만큼 꺼내주는 일꾼
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)
test_dataset = TensorDataset(X_test, y_test)

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)  # Train은 섞어야 좋음
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## 3. LSTM 모델 설계
**구조**: Embedding -> LSTM -> Linear (분류)

1. **Embedding**: 숫자(1, 23, 49...)를 의미 있는 벡터(숫자 묶음)로 바꿉니다. (컴퓨터가 단어의 의미를 학습하는 공간)
2. **LSTM**: 문장을 앞에서부터 읽으면서 문맥을 파악합니다.
3. **Linear (FC)**: 읽은 내용을 바탕으로 3개 클래스(그래픽, 야구, 우주) 중 어디에 속하는지 점수를 냅니다.

In [ ]:
import torch.nn as nn

class LSTMClassifier(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_size):
        super().__init__()
        # 1. 임베딩 층: 단어 개수(vocab_size) -> 벡터 크기(embedding_dim)로 변환
        # padding_idx=0: 0번(패딩)은 학습하지 않음 (의미 없으니까)
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        # 2. LSTM 층: 시계열 데이터 처리
        # batch_first=True: 입력 데이터가 (batch, seq_len, input_size) 형태임
        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
        
        # 3. 출력 층: 마지막 요약 정보를 보고 3개 카테고리로 분류
        self.fc = nn.Linear(hidden_size, 3)

    def forward(self, x):
        # x: (배치크기, 문장길이)
        x = self.embedding(x)       # (배치크기, 문장길이, 임베딩크기)
        
        # LSTM은 출력(output)과 마지막 상태(hidden, cell)를 반환함
        # 여기서 h는 (층개수, 배치크기, 은닉크기) 형태
        _, (h, c) = self.lstm(x)
        
        # 마지막 타임스텝의 은닉 상태(h[-1])를 가져와서 분류기에 넣음
        # 문장을 끝까지 다 읽은 후의 요약 정보라고 보면 됨
        out = self.fc(h[-1])
        return out

In [ ]:
# 4. 모델 학습 준비
import torch.optim as optim

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"사용 디바이스: {device}")

embedding_dim = 100
hidden_size = 128

model = LSTMClassifier(vocab_size, embedding_dim, hidden_size).to(device)
criterion = nn.CrossEntropyLoss() # 분류 문제의 채점표 (손실함수)
optimizer = optim.Adam(model.parameters(), lr=0.001) # 최적화 기법 (공부 방법)

# 학습 실행
epochs = 20 # 시간 관계상 20 에포크만 합시다.

for epoch in range(epochs):
    model.train() # 학습 모드 ON
    train_loss, train_acc = 0, 0
    train_total = 0
    train_correct = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()    # 이전 찌꺼기(기울기) 청소
        output = model(X_batch)  # 예측
        loss = criterion(output, y_batch) # 오차 계산
        loss.backward()          # 어디서 틀렸는지 역추적 (역전파)
        optimizer.step()         # 수정 (가중치 업데이트)

        train_loss += loss.item()
        # 정확도 계산
        pred = output.argmax(dim=1)
        train_correct += (pred == y_batch).sum().item()
        train_total += len(y_batch)
    
    # 에포크마다 리포트 출력
    avg_loss = train_loss / len(train_loader)
    avg_acc = train_correct / train_total
    print(f'Epoch {epoch+1}: Loss {avg_loss:.4f}, Accuracy {avg_acc:.4f}')

In [ ]:
# 5. 최종 평가 (Test Set)
from sklearn.metrics import classification_report

model.eval() # 평가 모드 (성적표 받는 시간)
all_preds, all_labels = [], []

with torch.no_grad(): # 평가할 때는 공부(gradient 계산) 안 함
    for X_batch, y_batch in test_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        output = model(X_batch)
        pred = output.argmax(dim=1)
        
        all_preds.extend(pred.cpu().numpy())
        all_labels.extend(y_batch.cpu().numpy())

print("--- 최종 성적표 ---")
print(classification_report(all_labels, all_preds, target_names=newsgroups.target_names))

---
## (심화) 사전 학습된 임베딩(Pre-trained Embedding) 사용하기
내가 처음부터 단어의 의미(임베딩)를 배우는 것보다, 미리 **영어 공부를 많이 한 모델(FastText)**의 지식을 빌려오면 성능이 더 좋아지지 않을까요?
- 마치 수능 영어를 칠 때, 알파벳부터 배우는 게 아니라 영단어장을 외우고 시작하는 것과 같습니다.

**Note**: `gensim` 라이브러리와 `ted_en_fasttext.model` 파일이 필요합니다.

In [ ]:
# gensim 설치 (없다면)
# %pip install gensim

In [ ]:
from gensim.models import FastText
import numpy as np

# 1. 미리 학습된 모델 로드 (가정: 같은 폴더에 모델 파일이 있다고 가정)
try:
    fasttext_model = FastText.load('ted_en_fasttext.model')
    print("사전 학습 모델 로드 성공!")
except:
    print("모델 파일이 없습니다. 이 부분은 스킵하거나 모델을 다운로드해야 합니다.")

# 2. 내 단어장(tokenizer)에 맞는 임베딩 행렬 만들기
if 'fasttext_model' in locals():
    embedding_dim = fasttext_model.vector_size
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    count = 0
    for word, i in tokenizer.word_index.items():
        if i >= vocab_size: break
        if word in fasttext_model.wv:
            embedding_matrix[i] = fasttext_model.wv[word]
            count += 1
    
    print(f"{vocab_size}개 단어 중 {count}개의 단어 벡터를 가져왔습니다.")

In [ ]:
class LSTMClassifierWithPretrained(nn.Module):
    def __init__(self, vocab_size, embedding_dim, embedding_matrix, hidden_size):
        super().__init__()
        # 1. 임베딩 층 선언
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        # 2. **중요**: 사전 학습된 가중치(지식) 덮어쓰기
        self.embedding.weight.data.copy_(torch.from_numpy(embedding_matrix))
        self.embedding.weight.requires_grad = True # False로 하면 지식 고정, True면 미세조정(Fine-tuning)

        self.lstm = nn.LSTM(embedding_dim, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, 3)

    def forward(self, x):
        x = self.embedding(x)
        _, (h, c) = self.lstm(x)
        out = self.fc(h[-1])
        return out

In [ ]:
# 모델 교체 후 재학습 (코드는 위와 동일하므로 생략하거나 재실행 가능)
# model = LSTMClassifierWithPretrained(...).to(device)
# ...